# Day comparison

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import sweetviz as sz
import pandas as pd
import pandas as pd
#from scapy.all import rdpcap
#from tueplots import bundles
#from tueplots.constants.color import rgb
from matplotlib.colors import LinearSegmentedColormap
#import chardet
import binascii
import os
import csv

In [9]:
# read them separately bcs they are huge and I get a memory overload
df_Mo=pd.read_csv('C:/Users/mihai/Documents/Faculta/Data Literacy/Mensa Project/csv_data/monday080124.csv')

C:\Users\mihai\AppData\Local\Temp\ipykernel_7696\4006711908.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('C:/Users/mihai/Documents/Faculta/Data Literacy/Mensa Project/csv_data/tuesday090124.csv')


AttributeError: 'list' object has no attribute 'to_csv'

In [ ]:
df_Tu=pd.read_csv('C:/Users/mihai/Documents/Faculta/Data Literacy/Mensa Project/csv_data/tuesday090124.csv')

In [ ]:
df_We=pd.read_csv('C:/Users/mihai/Documents/Faculta/Data Literacy/Mensa Project/csv_data/wednesday100124.csv')

In [ ]:
df_Th=pd.read_csv('C:/Users/mihai/Documents/Faculta/Data Literacy/Mensa Project/csv_data/Thursday110124.csv')

In [ ]:
df_Fr=pd.read_csv('C:/Users/mihai/Documents/Faculta/Data Literacy/Mensa Project/csv_data/Friday120124.csv')
df_all = [df_Mo, df_Tu, df_We, df_Th, df_Fr]

In [2]:
def str_to_hex(input_str):
    hex_str = binascii.hexlify(input_str.encode('latin-1')).decode()
    return hex_str

In [107]:
str_to_hex("eduroam")

'656475726f616d'

In [108]:
str_to_hex("UTguest")

'55546775657374'

In [109]:
str_to_hex("Guest")

'4775657374'

In [3]:
str_to_hex("<MISSING>")

'3c4d495353494e473e'

In [7]:
sum_src = []
sum_dest = []
sum_src_dest_all = []
#df_missing = []

for i, df in enumerate(df_all):
    df = df[(df["wlan.ssid"] == "656475726f616d")|( df["wlan.ssid"] == "55546775657374" )|( df["wlan.ssid"] == "4775657374")] #select all 3 uni networks
    #df_missing = df[df["wlan.ssid"] == '3c4d495353494e473e']
    
    df["frame.time"] = df["frame.time"].str.replace(r':\d+\.\d+', '', regex=True)
    df['frame.time'] = df['frame.time'].str.replace('Mitteleuropäische Zeit', 'CET')
    df['date-time'] = pd.to_datetime(df['frame.time'].str.strip(), format='%b %d, %Y %H:%M %Z', errors='coerce')
    df['hour:minute'] = df['date-time'].dt.strftime('%H:%M')
    df.dropna(subset=["wlan.sa"], inplace=True)  # Use inplace=True to modify the DataFrame in-place

    temp_src = df.groupby('hour:minute')['wlan.sa'].nunique()
    temp_dest = df.groupby('hour:minute')['wlan.da'].nunique()
    temp_sum = temp_src + temp_dest
    
    sum_src.append(temp_src)
    sum_dest.append(temp_src)
    sum_src_dest_all.append(temp_sum)

C:\Users\mihai\AppData\Local\Temp\ipykernel_20524\673582243.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["frame.time"] = df["frame.time"].str.replace(r':\d+\.\d+', '', regex=True)
C:\Users\mihai\AppData\Local\Temp\ipykernel_20524\673582243.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['frame.time'] = df['frame.time'].str.replace('Mitteleuropäische Zeit', 'CET')
C:\Users\mihai\AppData\Local\Temp\ipykernel_20524\673582243.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [8]:

sum_plus_10 = sum_src_dest_all * 1.1

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:

# Create a 5x2 grid of plots
fig, axes = plt.subplots(5, 2, figsize=(12, 16))

for i, df in enumerate(df_all):
    # Plot source on the left
    axes[i, 0].plot(sum_src[i], label='Source MAC')
    axes[i, 0].plot(sum_dest[i], label='Destination MAC')
    axes[i, 0].set_title(f'Day {i+1} - Source and Destination separately')
    
    # Plot sum_sa_da on the right
    axes[i, 1].plot(sum_src_dest_all[i], label='Sum Src and Dest MAC')
    axes[i, 1].plot(sum_plus_10[i], label='Scaled Sum Src and Dest')

    axes[i, 1].set_title(f'Day {i+1} - Sum Src and Dest + Scaling')
    
    # Add x-axis ticks and labels
    interval_minutes = 15
    tick_positions = np.arange(0, len(df.index), interval_minutes)
    axes[i, 0].set_xticks(tick_positions)
    axes[i, 0].set_xticklabels([sum_src.index[i] for i in tick_positions], rotation=45)
    axes[i, 1].set_xticks(tick_positions)
    axes[i, 1].set_xticklabels([sum_src.index[i] for i in tick_positions], rotation=45)
    
    # Add y-axis ticks
    axes[i, 0].set_yticks(range(0, 600, 50))
    axes[i, 1].set_yticks(range(0, 600, 50))
    
    # Add legend to the right subplot
    legend = axes[i, 1].legend(fontsize='small', loc='upper left', bbox_to_anchor=(1, 1))

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
# for df in df_all:
#     plt.plot(source, label='source MAC') # only max of 175
#     plt.plot(sum_sa_da_eduroam, label='sum source and dest only eduroam')
#     plt.plot(sum_sa_da, label='sum src and dest all 3')
#     plt.plot(sum_sa_da * 1.1, label = 'sum src and dest all 3 + 10%')
#     interval_minutes = 15
#     tick_positions = np.arange(0, len(sum_sa_da.index), interval_minutes)
#     plt.xticks(tick_positions, [sum_sa_da.index[i] for i in tick_positions], rotation=45)
#     plt.yticks(range(0,600,50))
#     plt.title('Count unique macs / minute')
#     legend = plt.legend(fontsize='small', loc='upper left', bbox_to_anchor=(1, 1))
#     plt.show()
#     #plt.xticks(unique_mac_per_min['date-time'].index)